# GraphQL APIs in Python

Trevor Olson

October 23, 2019

## What is GraphQL

### Describe your data

```graphql
type Meetup {
  name: String
  location: String
  talks: [Talk]
}
```

### Ask for what you want

```graphql
{
  meetup(name: "DesertPy") {
    location
  }
}
```

### Get predictable results

```json
{
  "meetup": {
    "location": "Phoenix, AZ"
  }
}
```

## GraphQL vs. REST: Pros

- Batching queries
- Easy to get only what you ask for
- Not attached to HTTP
- Streaming events

## GraphQL vs. REST: Cons

- Caching
- File uploading/downloading
- 

## Try it out

- [Github](https://developer.github.com/v4/)
- [Yelp](https://www.yelp.com/developers/graphql/guides/intro)
- [Countries](https://github.com/trevorblades/countries)

## But let's talk about Python

- [Ariadne](https://ariadnegraphql.org/)
- [Graphene](https://graphene-python.org/)

## Graphene

```bash
> pip install graphene
```

### Works with...

- [Graphene-Django](http://docs.graphene-python.org/projects/django/en/latest/)
- [Flask-Graphql](https://github.com/graphql-python/flask-graphql)
- [Graphene-SQLAlchemy](http://docs.graphene-python.org/projects/sqlalchemy/en/latest/)
- [Graphene-GAE](https://github.com/graphql-python/graphene-gae/)
- [Graphene-Mongo](https://graphene-mongo.readthedocs.io/en/latest/)
- [Starlette](https://www.starlette.io/graphql/)
- [FastAPI](https://fastapi.tiangolo.com/tutorial/graphql/)

### Define the schema

In [ ]:
import graphene

class Person(graphene.ObjectType):
    name = graphene.String(description='The name of this person.')
    height = graphene.Float(
        description='The height of this person in meters.')
    mass = graphene.Float(
        description='The mass of this person in kilograms.')
    created = graphene.DateTime(
        description='The datetime that this resource was edited.')

### Resolve relations

In [ ]:
class Planet(graphene.ObjectType):
    name = graphene.String(description='The name of this planet.')
    diameter = graphene.String(
        description='The diameter of this planet in kilometers.')

class Person(graphene.ObjectType):
    # ...
    homeworld = graphene.Field(
        Planet, description='The planet that this person was born on.')
    
    def resolve_homeworld(self, info):
        return get_planet_by_id(self.homeworld)

### Root query

In [ ]:
class Query(graphene.ObjectType):
    planet = graphene.Field(Planet, name=graphene.String(required=True))
    person = graphene.Field(Person, name=graphene.String(required=True))

    def resolve_planet(self, info, name):
        return get_planet_by_name(name)
    
    def resolve_person(self, info, name):
        return get_person_by_name(name)

### Attach the endpoint

In [ ]:
from starlette.applications import Starlette
from starlette.graphql import GraphQLApp

app = Starlette()
app.add_route('/', GraphQLApp(schema=graphene.Schema(query=Query)))

In [ ]:
from IPython.display import IFrame
IFrame('http://127.0.0.1:5000', width='100%', height=768)

### Mutations

In [ ]:
class MarkFavoritePerson(graphene.Mutation):
    class Arguments:
        name = graphene.String(required=True)
        favorite = graphene.Boolean(required=True)

    Output = Person

    def mutate(root, info, name, favorite):
        person = get_person_by_name(name)
        mark_person_as_favorite(person, favorite)
        return person

```graphql
mutation {
    markFavoritePerson(name: "Leia Organa", favorite: true) {
        name
        hight
        birthYear
    }
}
```

### Batching

In [ ]:
from promise import Promise
from promise.dataloader import DataLoader

class PlanetLoader(DataLoader):
    def batch_load_fn(self, keys):
        # Here we return a promise that will result on the
        # corresponding user for each key in keys
        return Promise.resolve([get_planet_by_name(key) for key in keys])
    
planet_loader = PlanetLoader()
planet_loader.load("Alderaan").then(lambda planet: do_something(planet))

### Async executors

In [ ]:
from graphql.execution.executors.asyncio import AsyncioExecutor

from starlette.applications import Starlette
from starlette.graphql import GraphQLApp

app = Starlette()
app.add_route('/', GraphQLApp(
    schema=graphene.Schema(query=Query), executor_class=AsyncioExecutor))

## Thanks!

https://graphql.org/

https://graphene-python.org/

https://www.starlette.io/graphql/

In [ ]:
from IPython.display import Code
Code(filename='server.py')